# Example of DOV search methods for bodem objects

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2Fsearch_boringen.ipynb)

## Use cases explained below
* Introduction to the bodem-objects
* Get bodemsites in a bounding box
* Get bodemlocaties with specific properties
* Get all direct and indirect bodemobservaties in bodemlocatie


In [1]:
%matplotlib inline
import inspect, sys
import warnings; warnings.simplefilter('ignore')

In [2]:
# check pydov path
import pydov

## Get information about the datatype 'Bodemlocatie'

Other datatypes are also possible:
* Bodemsite: BodemsiteSearch
* Bodemmonster: BodemmonsterSearch
* Bodemobservatie: BodemobservatieSearch

In [3]:
from pydov.search.bodemlocatie import BodemlocatieSearch
bodemlocatie = BodemlocatieSearch()

A description is provided for the 'Bodemlocatie' datatype:

In [4]:
bodemlocatie.get_description()

"Een bodemlocatie is ofwel een profielput of een boring. Een boring is altijd één puntlocatie (x,y,z) en een profielput heeft minimum één en maximum twee puntlocaties (begin- en eindpunt van de profielput). \r\nEen profielput is een uitgegraven put in de bodem waarin profielbeschrijvingen, monsternames of bodemobservaties worden uitgevoerd. Een profielbeschrijving is een waarneming van bodemhorizonten en/of bodemlagen in een uitgegraven profielput. Een bodemhorizont is een visueel te onderscheiden deel van de bodem dat ontstaan is door omzetting van het moedermateriaal door pedogenetische processen of door het afzetten van organisch materiaal. Een bodemhorizont heeft voor de meeste bodemvariabelen homogene morfologische en analytische karakteristieken. Een bodemlaag daarentegen is ontstaan door niet-pedogenetische processen. Aan de hand van een profielput krijg je een beeld van de bodemkundige opbouw.\r\nEen boring is het resultaat van het boren in de ondergrond met verwijdering van bo

The different fields that are available for objects of the 'Bodemlocatie' datatype can be requested with the get_fields() method:

In [5]:
fields = bodemlocatie.get_fields()

# print available fields
for f in fields.values():
    print(f['name'])

naam
pkey_bodemlocatie
type
rapport_bodemlocatie
profielbeschrijving
datum
doel
x
y
z
Auteurs
Aantal_classificaties
Aantal_opbouwen
erfgoed
Aantal_observaties
Aantal_monsters
bodemstreek
Bodemsite
Opdrachten
waarnemingsdatum
invoerdatum
educatieve_waarde


You can get more information of a field by requesting it from the fields dictionary:
* *name*: name of the field
* *definition*: definition of this field
* *cost*: currently this is either 1 or 10, depending on the datasource of the field. It is an indication of the expected time it will take to retrieve this field in the output dataframe.
* *notnull*: whether the field is mandatory or not
* *type*: datatype of the values of this field

In [6]:
fields['type']

{'name': 'type',
 'definition': 'Het type van de bodemlocatie: boring of profielput.',
 'type': 'string',
 'notnull': False,
 'query': True,
 'cost': 1,
 'values': {'boring': 'Een boring is het resultaat van het boren in de ondergrond met verwijdering van bodem door middel van een gereedschap in de vorm van een holle buis. Aan de hand van dit opgeboorde bodemmateriaal worden bodembeschrijvingen, bodemobservaties en monsternames uitgevoerd.',
  'profielput': 'Een profielput is een uitgegraven put in de bodem waarin profielbeschrijvingen, monsternames of bodemobservaties worden uitgevoerd. Een profielbeschrijving is een waarneming van bodemhorizonten en/of bodemlagen in een uitgegraven profielput. Een bodemhorizont is een visueel te onderscheiden deel van de bodem dat ontstaan is door omzetting van het moedermateriaal door pedogenetische processen of door het afzetten van organisch materiaal. Een bodemhorizont heeft voor de meeste bodemvariabelen homogene morfologische en analytische kar

Optionally, if the values of the field have a specific domain the possible values are listed as *values*:

In [7]:
fields['type']['values']

{'boring': 'Een boring is het resultaat van het boren in de ondergrond met verwijdering van bodem door middel van een gereedschap in de vorm van een holle buis. Aan de hand van dit opgeboorde bodemmateriaal worden bodembeschrijvingen, bodemobservaties en monsternames uitgevoerd.',
 'profielput': 'Een profielput is een uitgegraven put in de bodem waarin profielbeschrijvingen, monsternames of bodemobservaties worden uitgevoerd. Een profielbeschrijving is een waarneming van bodemhorizonten en/of bodemlagen in een uitgegraven profielput. Een bodemhorizont is een visueel te onderscheiden deel van de bodem dat ontstaan is door omzetting van het moedermateriaal door pedogenetische processen of door het afzetten van organisch materiaal. Een bodemhorizont heeft voor de meeste bodemvariabelen homogene morfologische en analytische karakteristieken. Een bodemlaag daarentegen is ontstaan door niet-pedogenetische processen. Aan de hand van een profielput krijg je een beeld van de bodemkundige opbouw

## Example use cases

### Get bodemsites in a bounding box

Get data for all the bodemsites that are geographically located completely within the bounds of the specified box.

The coordinates are in the Belgian Lambert72 (EPSG:31370) coordinate system and are given in the order of lower left x, lower left y, upper right x, upper right y.

The same methods can be used for other bodem-objects.

In [8]:
from pydov.search.bodemsite import BodemsiteSearch
bodemsite = BodemsiteSearch()

In [9]:
from pydov.util.location import Within, Box

df = bodemsite.search(location=Within(Box(148000, 160800, 160000, 169500)))
df.head()

[000/007] ccccccc


,pkey_bodemsite,naam,datum,beschrijving,waarnemingsdatum,invoerdatum
0,https://www.dov.vlaanderen.be/data/bodemsite/2...,Zoniënwoud_Groenendaal,NaN,"Zachte helling op plateau, Bodem nooit onder l...",NaN,NaN
1,https://www.dov.vlaanderen.be/data/bodemsite/2...,Zoniënwoud_Rood Klooster,NaN,Zeer zachte helling op plateau,NaN,NaN
2,https://www.dov.vlaanderen.be/data/bodemsite/1...,Zoniënwoud_Tertiair,1983-01-01,NNW gerichte helling van een 10% langs vallei ...,1983-01-01,NaN
3,https://www.dov.vlaanderen.be/data/bodemsite/2...,Zoniënwoud_Sint_Hubertusdreef,NaN,"Hoofd van een concave droge vallei (del), depr...",NaN,NaN
4,https://www.dov.vlaanderen.be/data/bodemsite/2...,Watermaal-Bosvoorde_Zoniënwoud,2011-06-14,bos,2011-06-14,NaN


The dataframe contains a list of bodemsites. The available data are flattened to represent unique attributes per row of the dataframe.

Using the *pkey_bodemsite* field one can request the details of this bodemsite in a webbrowser:

In [10]:
for pkey_bodemsite in set(df.pkey_bodemsite):
    print(pkey_bodemsite)

https://www.dov.vlaanderen.be/data/bodemsite/2017-000215
https://www.dov.vlaanderen.be/data/bodemsite/1983-000216
https://www.dov.vlaanderen.be/data/bodemsite/2011-000247
https://www.dov.vlaanderen.be/data/bodemsite/2017-000305
https://www.dov.vlaanderen.be/data/bodemsite/2017-000229
https://www.dov.vlaanderen.be/data/bodemsite/2017-000210
https://www.dov.vlaanderen.be/data/bodemsite/2017-000306


### Get bodemlocaties with specific properties

Next to querying bodem objects based on their geographic location within a bounding box, we can also search for bodem objects matching a specific set of properties. 
The same methods can be used for all bodem
For this we can build a query using a combination of the 'Bodemlocatie' fields and operators provided by the WFS protocol.

A list of possible operators can be found below:

In [11]:
[i for i,j in inspect.getmembers(sys.modules['owslib.fes'], inspect.isclass) if 'Property' in i]

['PropertyIsBetween',
 'PropertyIsEqualTo',
 'PropertyIsGreaterThan',
 'PropertyIsGreaterThanOrEqualTo',
 'PropertyIsLessThan',
 'PropertyIsLessThanOrEqualTo',
 'PropertyIsLike',
 'PropertyIsNotEqualTo',
 'PropertyIsNull',
 'SortProperty']

In this example we build a query using the *PropertyIsEqualTo* operator to find all bodemlocaties with bodemstreek 'zandstreek'.
We use *max_features=10* to limit the results to 10.

In [12]:
from owslib.fes import PropertyIsEqualTo

query = PropertyIsEqualTo(propertyname='bodemstreek',
                          literal='Zandstreek')
df = bodemlocatie.search(query=query, max_features=10)

df.head()

[000/010] cccccccccc


,pkey_bodemlocatie,naam,type,datum,doel,x,y,z,erfgoed,bodemstreek,waarnemingsdatum,invoerdatum,educatieve_waarde
0,https://www.dov.vlaanderen.be/data/bodemlocati...,KULEU_30,profielput,2012-05-14,bodemkundige opbouw,160992.14,196155.14,5.00,True,Zandstreek,2012-05-14,2012-05-14,ZEER
1,https://www.dov.vlaanderen.be/data/bodemlocati...,INBO_2,profielput,2007-05-04,bodemkundige opbouw,56598.44,196314.35,20.00,True,Zandstreek,2007-05-04,2007-05-04,OK
2,https://www.dov.vlaanderen.be/data/bodemlocati...,KULEU_15,profielput,2010-06-06,bodemkundige opbouw,175001.18,177459.73,65.58,True,Zandstreek,2010-06-06,2010-06-06,ZEER
3,https://www.dov.vlaanderen.be/data/bodemlocati...,BAAC_12,profielput,2012-07-31,archeologische landschappelijke profielputten,155631.46,184083.61,11.00,True,Zandstreek,2012-07-31,2012-07-31,ZEER
4,https://www.dov.vlaanderen.be/data/bodemlocati...,ILVO_1,profielput,2012-11-20,bodemkundige opbouw,108115.73,185850.71,32.00,True,Zandstreek,2012-11-20,2012-11-20,ZEER


Once again we can use the *pkey_bodemlocatie* as a permanent link to the information of these bodemlocaties:

In [13]:
for pkey_bodemlocatie in set(df.pkey_bodemlocatie):
    print(pkey_bodemlocatie)

https://www.dov.vlaanderen.be/data/bodemlocatie/2014-000039
https://www.dov.vlaanderen.be/data/bodemlocatie/2013-000048
https://www.dov.vlaanderen.be/data/bodemlocatie/2012-000041
https://www.dov.vlaanderen.be/data/bodemlocatie/2010-000014
https://www.dov.vlaanderen.be/data/bodemlocatie/2012-000026
https://www.dov.vlaanderen.be/data/bodemlocatie/2007-000102
https://www.dov.vlaanderen.be/data/bodemlocatie/2012-000027
https://www.dov.vlaanderen.be/data/bodemlocatie/2012-000033
https://www.dov.vlaanderen.be/data/bodemlocatie/2014-000040
https://www.dov.vlaanderen.be/data/bodemlocatie/2012-000028


### Get all direct and indirect bodemobservaties in bodemlocatie

Get all bodemobservaties in a specific bodemlocatie.
Direct means bodemobservaties directly linked with a bodemlocatie.
Indirect means bodemobservaties linked with child-objects of the bodemlocatie, like bodemmonsters.

In [14]:
from pydov.search.bodemobservatie import BodemobservatieSearch
from pydov.search.bodemlocatie import BodemlocatieSearch
bodemobservatie = BodemobservatieSearch()
bodemlocatie = BodemlocatieSearch()

In [15]:
from owslib.fes import PropertyIsEqualTo
from pydov.util.query import Join

bodemlocaties = bodemlocatie.search(query=PropertyIsEqualTo(propertyname='naam', literal='VMM_INF_52'), 
                                   return_fields=('pkey_bodemlocatie',))

bodemobservaties = bodemobservatie.search(query=Join(bodemlocaties, 'pkey_bodemlocatie'))
bodemobservaties.head()

[000/021] ccccccccccccccccccccc


,pkey_bodemobservatie,pkey_bodemlocatie,Aan,pkey_parent,parameter,parametergroep,waarde,eenheid,ondergrens,bovengrens,methode,betrouwbaarheid,veld_labo,diepte_van,diepte_tot,observatiedatum,invoerdatum
0,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,bodemmonster,https://www.dov.vlaanderen.be/data/bodemmonste...,Textuurfracties,Bodem_fysisch_textuur,Fractiemetingen,%,NaN,NaN,Textuur pipetmethode Robinson-Köhn,goed,labo,40.0,85.0,2015-09-08,2019-02-19
1,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,bodemdiepteinterval,https://www.dov.vlaanderen.be/data/bodemdiepte...,Ksat,Bodem_fysisch_vocht,0.0000072879,m/s,NaN,NaN,Ksat Open-end-methode,goed,veld,40.0,85.0,2015-09-08,2019-02-07
2,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,bodemdiepteinterval,https://www.dov.vlaanderen.be/data/bodemdiepte...,Ksat,Bodem_fysisch_vocht,0.0000156761,m/s,NaN,NaN,Ksat Porchet-methode,goed,veld,40.0,85.0,2015-09-08,2019-02-07
3,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,bodemdiepteinterval,https://www.dov.vlaanderen.be/data/bodemdiepte...,Ksat,Bodem_fysisch_vocht,0.000002583,m/s,NaN,NaN,Ksat Enkele-ring-methode,goed,veld,40.0,85.0,2015-09-08,2019-02-07
4,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,bodemdiepteinterval,https://www.dov.vlaanderen.be/data/bodemdiepte...,Ksat,Bodem_fysisch_vocht,0.0000029494,m/s,NaN,NaN,Ksat Porchet-methode,goed,veld,40.0,85.0,2015-09-08,2019-02-07


### Get all bodemobservaties in bodemmonster

Get all bodemobservaties linked with a bodemmonster

In [16]:
from pydov.search.bodemmonster import BodemmonsterSearch
bodemmonster = BodemmonsterSearch()

bodemmonsters = bodemmonster.search(query=PropertyIsEqualTo(propertyname = 'identificatie', literal='A0057359'),
                                   return_fields=('pkey_bodemmonster',))

bodemobservaties = bodemobservatie.search(query=Join(bodemmonsters, on = 'pkey_parent', using='pkey_bodemmonster'))
bodemobservaties.head()

[000/003] ccc


,pkey_bodemobservatie,pkey_bodemlocatie,Aan,pkey_parent,parameter,parametergroep,waarde,eenheid,ondergrens,bovengrens,methode,betrouwbaarheid,veld_labo,diepte_van,diepte_tot,observatiedatum,invoerdatum
0,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,bodemmonster,https://www.dov.vlaanderen.be/data/bodemmonste...,Textuurfracties,Bodem_fysisch_textuur,Fractiemetingen,%,NaN,NaN,Textuur pipetmethode Robinson-Köhn,goed,labo,40.0,85.0,2015-09-08,2019-02-19
1,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,bodemmonster,https://www.dov.vlaanderen.be/data/bodemmonste...,Textuur - granulometrisch - klassen bodemkarte...,Bodem_fysisch_textuur,E - Klei,-,NaN,NaN,Textuur pipetmethode Robinson-Köhn,goed,labo,40.0,85.0,2015-08-27,2019-02-07
2,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,bodemmonster,https://www.dov.vlaanderen.be/data/bodemmonste...,Textuur - handmatig - klassen bodemkartering,Bodem_fysisch_textuur,A - Leem,-,NaN,NaN,Textuur manuele schatting (palpatie),goed,labo,40.0,85.0,2015-08-27,2019-02-07
